In [17]:
from imutils import paths
import cv2
import os
from recognition_modules.pre_process.pre_process import pre_process_frame
from recognition_modules.feature_extractor.ageitgey_model import get_embeddings
from recognition_modules.face_match.election_based import get_name_election

In [18]:
# load paths
paths= list(paths.list_images("Test/"))
# load detector
detector = cv2.CascadeClassifier('detector.xml')

In [19]:
data = {
        'names':[],
        'encodings':[]
    }

In [20]:
def improve_contrast_image_using_clahe(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    hsv_planes = cv2.split(hsv)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    hsv_planes[2] = clahe.apply(hsv_planes[2])
    hsv = cv2.merge(hsv_planes)
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

In [21]:
for path in paths_encodings:
    try:
        # get the frame
        frame = cv2.imread(path)
        # pre process frame
        rects = pre_process_frame(frame,detector)
        x1, y1, width, height = rects
        x2, y2 = x1 + width, y1 + height
        face = frame[y1:y2, x1:x2]
        face = improve_contrast_image_using_clahe(face)
        # get embeddings
        encoding = get_embeddings(rects,face)
        # save in the dictionary
        data['names'].append(path.split(os.path.sep)[-2])
        data['encodings'].append(encoding)
    except:
        print(f'{path} com problema')

Test/Diego/Diego_2.jpeg com problema
Test/Otavio/Otavio_2.jpeg com problema
Test/Caroline_Carvalho/Carol_1.jpg com problema
Test/Caroline_Carvalho/Carol_2.jpg com problema
Test/Juliana_Puterman/Juliana_Puterman_2.jpeg com problema
Test/Cesar/Cesar_5.jpg com problema
Test/Felipe/Felipe_4.jpg com problema
Test/Random_71/images (1).jpeg com problema
Test/Random_71/elle-macpherson-today-main-190606_10b538ae0c490cc6eafc09bad599f12c.jpg com problema
Test/Random_49/Ron-Weasley-e1436589589658.jpg com problema
Test/Bia/Bia_2.jpeg com problema
Test/Bia/Bia_5.jpeg com problema
Test/Random_40/CAROL2-1.jpg com problema
Test/Random_47/thumbnail-for-90895.jpg com problema
Test/Random_47/Entrevista_Dezembro_de_2018_-_Mel_Maia,_02.jpg com problema
Test/Vinicius_Almeida/Vini_4.jpg com problema
Test/Elizabete/Elizabete_5.jpg com problema
Test/Random_14/00000001.jpg com problema
Test/Random_22/tony-ramos.jpg com problema
Test/Random_25/download (1).jpeg com problema
Test/Random_25/Laura-Cardoso-como-Matil

In [22]:
import pandas as pd
data = pd.DataFrame(data)
length = data.groupby('names')['names'].agg('count')
data_count = pd.DataFrame(
    {
        'names':length.index.to_list(),
        'tamanho':length.to_list()
    }
)
data_final = data.set_index('names').join(data_count.set_index('names'))
data_final = data_final[data_final['tamanho'] >= 2]
data_final = data_final.reset_index()
data_final = data_final[['names','encodings']]
data = {
    'names':data_final['names'].to_list(),
    'encodings':data_final['encodings'].to_list()
}

In [23]:
import pickle
with open('encodings_clahe_face.pickle', 'wb') as handle:
    pickle.dump(data, handle)